# Hopenhayn and Rogerson (1993) 

This notebook provides my code to find the stationary equilibrium of HR model.

Calibration is based on my note.


Since we don't consider adjustment cost, a state variable is only productivity shock.


- Main function: Compute_HR
- struct: Model, contains params

In [1]:
struct Model
    
    # Hopenhayn and Rogerson 1993
    # store  model parameters
    β::Float64 # discount factor
    θ::Float64 # labor share
    A::Float64 # disutility from labor (- aN_t)
    cf::Float64 # fixed operation cost
    ce::Float64 # one time fixed entry cost
    sgrid::Array{Float64,1} ## productivy shock grid
    ns::Int64 # number of shock grid
    F::Array{Float64,2} ## transition probability
    ν::Array{Float64,1} # entry distrinution
    ndemand::Function
    f::Function
    maxiter::Int64 # maximum number of iteration
    tol::Float64 # tolerance
end

In [2]:
function Construct()
    
    #primitive params 
    
    β = 0.8 # discount factor
    θ = 0.64 # labor share
    sgrid = [3.98e-4, 3.58, 6.82, 12.18, 18.79] # productivity shock grid
    ns = length(sgrid) # number of shock grid
    F = [0.6598 0.2600 0.0416 0.0331 0.0055;
         0.1997 0.7201 0.0420 0.0326 0.0056;
         0.2000 0.2000 0.5555 0.0344 0.0101;
         0.2000 0.2000 0.2502 0.3397 0.0101;
         0.2000 0.2000 0.2500 0.3400 0.0100] # transition matrix
    ν = [0.37, 0.4631, 0.1102, 0.0504, 0.0063] # invariant distribution
    # Here invariant distribution is given.
    # However, we can obtain this value using F.


    ndemand(p,s,θ)=(θ*p*s)^(1.0/(1.0-θ)) # labor demand function via FOC
    f(s,n,θ)= s*(n^θ) # production function
    A = 1.0/200.0 # parameter that determine disutility from labor 
    cf = 10.0 # fixed cost for operation
    ce = 15.0 # fixed cost for entry
    
    maxiter = 10000 # maximum number of iteration
    tol =1e-5 # tolerance
    
    return model = Model(β, θ, A, cf, ce, sgrid, ns, F, ν, ndemand, f, maxiter, tol)
end

Construct (generic function with 1 method)

In [3]:
function ComputeHR(m::Model)
    
    """
    This function is a main function of Hopenhayn and Rogerson(1993).
    
    Arguments:
    
    m: struct that contains parameters and iteration settings.
    
    Return:
    
    p:    Equilibrium price, Float64
    dlab: Labor demand, Float64
    slab: Labor supply, Float64
    μ:   Stationary distribution for firms, Array{Float64,1}
    M:    Mass of entrants, Float64
    """

    # Step 1. Compute equilibrium price
    
    p,X = Find_price(m)
    
    # Step 2. Find invariant distribution and aggregate labor
    
    dlab::Float64=0.0
    slab::Float64=0.0
    
    dlab,slab,μ,M = Find_labor(m,p,X) 
    
    return p,dlab,slab,μ, M
end

ComputeHR (generic function with 1 method)

In [4]:
function Find_price(m::Model)
    """
    This function computes equilibrium price by bisection method.
    
    Arguments:
    
    m: Struct that contains parameters and iteration settings

    
    Return:
    p: Equilibrium price, Float64
    W: Converged value function, Array{Float64,1}
    X: Policy function for entry-exit decision, Array{Float64,1}
    """
    
    p_lower = 0.0 # lower bound for p
    p_upper = 100.0 # upper bound for p
    
    p = 0.5*p_lower+ 0.5*p_upper
  
    # preallocate 
    X = zeros(m.ns)
    W = zeros(m.ns)


    @inbounds for price_iter in 1:m.maxiter

        # Find decision rules and value functions

        W,X= VFI_incumbents(m,p) 
        We = Bellman_entrant(m,p,W)
       
        # Check free entry condition.

        FEC = Check_free_entry(m,p,We)
        
        # Check free entry condition and update price
        
        if abs(FEC) < m.tol
            break
        end

        if FEC > 0.0
            p_upper = p # lower price

        elseif FEC < 0.0
            p_lower = p　# raise price
        end
    
        # update price     
        p = 0.5*p_lower + 0.5*p_upper

    end
    
    return p, X,W
end

Find_price (generic function with 1 method)

In [5]:
function VFI_incumbents(m::Model, p)
    """
    This function does value function iteration for the value function of incumbents
    
    Arguments:
    
    m: Struct that contains parameters and iteration settings
    p: price, Float64
    
    Return:
    
    W: Converged value function, Array{Float64,1}
    X: Policy function for entry-exit decision

    """

    W = zeros(m.ns) # initial guess for value function
    X = zeros(m.ns) # store entry-exit policy function

    @inbounds for VFI_iter in 1:m.maxiter
        
        TW, X = Bellman_incumbent(m,p,W)     
        err = maximum(abs.(TW-W))
        W = copy(TW)
        
        if err < m.tol
            break
        end
    end
        
    return W, X
end

VFI_incumbents (generic function with 1 method)

In [6]:
function Bellman_incumbent(m::Model,p, W)
    
    """
    This function computes value function of incumbents
    
    Arguments:
    
    m: Struct that contains parameters and iteration settings
    p: price, Float64
    W: Guessed value function of incumbents, Array{Float64,1}
    
    Return: 
    TW: Updated value function, Array{Float64,1}
    X :  Policy function for entry-exit decision
    
    """
    
    TW = zeros(m.ns)  # store value
    X  = zeros(m.ns) # store optimal decision about entry and exit 
  
    @inbounds for (i_s, v_s) in enumerate(m.sgrid)
        
        profit::Float64 = p*m.f(v_s,m.ndemand(p,v_s,m.θ),m.θ)- m.ndemand(p,v_s,m.θ) - p*m.cf
        @views stay_val::Float64 = profit + m.β*dot(m.F[i_s,:],W)
      
        if stay_val >= profit
            TW[i_s] = stay_val
        else
            TW[i_s] = profit
            X[i_s] = 1.0
        end   
    end
    
    return TW, X
end

Bellman_incumbent (generic function with 1 method)

In [7]:
function Bellman_entrant(m::Model,p,W)
    """
    This functiom compute value function of entrants using that of incumbents
    
    Arguments:
    
    m: Struct that contains parameters and iteration settings
    p: price, Float64
    W: Value function of incumbents, Array{Float64,1}
    
    Return: 
    We: Value function for entrants type:Array{Float64,1}
    
    """
    We = zeros(m.ns)
    @inbounds for (i_s, v_s) in enumerate(m.sgrid)
        profit::Float64 = p*m.f(v_s,m.ndemand(p,v_s,m.θ),m.θ)- m.ndemand(p,v_s,m.θ)
        We[i_s] += profit +m.β*dot(m.F[i_s,:],W)
    end
    return We
end  

Bellman_entrant (generic function with 1 method)

In [8]:
function Check_free_entry(m::Model,p,We)
    """
    This function returns  residual of free entry condition
    
    Arguments:
    
    m: Struct that contains parameters and iteration settings
    p: price, Float64
    We: Value function of entrants, Array{Float64,1}
    
    Return:
    
    resitual of free entry condition
    
    """
    return dot(m.ν,We)- p*m.ce
end

Check_free_entry (generic function with 1 method)

In [9]:
function Find_labor(m,p,X)
    
    """
    This function compute invariant distribution and aggregate labor.
    Iteration based.
    
    Arguments:
    
    m: Struct that contains parameters and iteration settings
    p: Equilibrium price, Float64
    X: Policy function for entry-exit decision, Array{Float64,1}
    
    
    Return:
    
    dlab: labor demand, Float64
    slab: labor supply, Float64
    μ: stationary distribution, Array{Float64, 1} 
    M: mass of entrants, Float64
    """
    
    const θ = m.θ # unpack params
    
    # Initial guess for mass of entrants.
    M = 1.0
    # Initial guess for distribution
    μ = ones(m.ns)./m.ns
    
    # Preallocate distribution
    μ1 = zeros(m.ns)
    
    # Store labor data 
    dlab = 0.0
    slab::Float64 = 0.0
    
    @inbounds for labor_iter in 1:m.maxiter

        # Compute Stationary distribution 

        @inbounds for dist_iter in 1:m.maxiter

            @inbounds for (i_sprime,v_sprime) in enumerate(m.sgrid)
                @inbounds for (i_s, v_s) in enumerate(m.sgrid)
                    μ1[i_sprime] +=
                    (1.0 - X[i_s])*m.F[i_s,i_sprime]*(μ[i_s] + M*m.ν[i_s])
                end
            end
            
            dist_err::Float64 = maximum(abs.(μ-μ1))
            μ = copy(μ1)
            μ1 = zeros(m.ns)
            
            if dist_err < m.tol
                break
            end
            
            if dist_iter == m.maxiter
                println("distribution does not converge")
            end
        end
            
        # Derive labor demand    
        dlab::Float64 =0.0
        @inbounds for (i_s,v_s) in enumerate(m.sgrid)
            dlab += m.ndemand(p,v_s,θ)*μ[i_s] 
            dlab += M*m.ndemand(p,v_s,θ)*m.ν[i_s]
        end
              
        # Compute aggregate profit
        profit::Float64 = 0.0
        @inbounds for (i_s,v_s) in enumerate(m.sgrid)
            gross::Float64 =p*m.f(v_s,m.ndemand(p,v_s,θ),θ)-  m.ndemand(p,v_s,θ)
            profit += (gross - p*m.cf)*μ[i_s] # profit for incumbents
            profit += M*(gross - p*m.ce)*m.ν[i_s] # profit for entrants
        end
        
        slab = (1.0/m.A) - profit
              
        # Check market clearing condition
        
        LMC::Float64 = dlab - slab
        
        if abs(LMC) < m.tol
            break
        end
        
        # update mass of entrants

        if LMC > 0.0
            M = 0.999*M  # lower M
        else
            M = 1.001*M # raise M
        end
        
        if labor_iter == m.maxiter
            println("aggregate labor does not converge")
        end
    end
        
    return dlab,slab,μ,M
end

Find_labor (generic function with 1 method)

In [10]:
model = Construct()

Model(0.8, 0.64, 0.005, 10.0, 15.0, [0.000398, 3.58, 6.82, 12.18, 18.79], 5, [0.6598 0.26 … 0.0331 0.0055; 0.1997 0.7201 … 0.0326 0.0056; … ; 0.2 0.2 … 0.3397 0.0101; 0.2 0.2 … 0.34 0.01], [0.37, 0.4631, 0.1102, 0.0504, 0.0063], ndemand, f, 10000, 1.0e-5)

In [11]:
p,dlab,slab, μ,M = ComputeHR(model)

(0.7422052323818207, 188.36077772174045, 188.36077869786038, [1.72036, 4.95924, 1.36829, 0.504264, 0.0589574], 2.730743276114718)

In [12]:
price,X,W =Find_price(model)

(0.7422052323818207, [1.0, 0.0, 0.0, 0.0, 0.0], [-7.42205, -4.26049, 18.477, 96.5738, 268.725])

In [13]:
dlab, slab, μ,M= Find_labor(model,p,X)

(188.36077772174045, 188.36077869786038, [1.72036, 4.95924, 1.36829, 0.504264, 0.0589574], 2.730743276114718)

In [14]:
@time ComputeHR(model);

  0.152382 seconds (1.38 M allocations: 66.655 MiB, 5.58% gc time)
